In [10]:
%load_ext autoreload
%autoreload 2

import ast
from pathlib import Path
from pprint import pprint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from docstring_format import *
from docstring_format.base import *
from docstring_format.constants import *
import json

In [2]:
# file = Path('equation_parser/functions.py')
file = Path('./tests/dummy_tests_functions.py')

raw_text = file.read_text()
dirty_lines = raw_text.splitlines()
tree = ast.parse(raw_text)

classes = [item for item in tree.body if isinstance(item, ast.ClassDef)]
class_methods = [func for item in classes for func in item.body if isinstance(func, ast.FunctionDef)]
functions = [item for item in tree.body if isinstance(item, ast.FunctionDef)]

In [7]:
from dataclasses import dataclass

In [48]:
@dataclass
class Sections:
    summary_text: str
    param_delimiter: str
    parameters_text: str
    return_delimiter: str
    returns_text: str

@dataclass
class Parameter:
    name: str
    annotation: str
    
@dataclass
class Docstring:
    function: ast.FunctionDef
    arguments: list[str]
    docstring_text: str
    sections: Sections
    start: int
    length: int

In [49]:
def get_docstring_sections(docstring, style: DocstringStyle = DocstringStyle.NUMPY) -> Optional[DocstringSection]:
    """Parse the docstring to get sections.

    Sections are defined as :
     - summary (everything until the parameters definition).
     - a parameter param_delimiter (depending of the writing style of the docstring)
     - the parameters"""
    param_token = PARAMETERS_DELIMITERS_REGEX[style]
    return_token = RETURNS_DELIMITERS_REGEX[style]
    pattern = re.compile('(?P<summary_text>.*)'
                         f'(?P<param_delimiter>{param_token})'
                         f'(?P<parameters_text>((?!{return_token}).)*)'  # negative lookahead of return token
                         f'(?P<return_delimiter>{return_token})?'
                         f'(?P<returns_text>.*)?',
                         flags=re.S)
    match = re.search(pattern, docstring)
    if match:
        return Sections(**match.groupdict())

[Parameter(name='arg1', annotation='str')]

In [52]:
docstrings = []
for func in functions:
    start, length = get_docstring_lines(func, dirty_lines)
    docstring = get_docstring_from_position(dirty_lines, start, length)
    sections = get_docstring_sections(docstring)
    
    docstrings.append(
        Docstring(function=func, 
                  docstring_text=docstring, 
                  start=start,
                  length=length, 
                  sections=sections,
                  arguments=[Parameter(name=arg.arg, annotation=parse_annotation(arg)) for arg in func.args.args])
    )

In [53]:
docstrings

[Docstring(function=<ast.FunctionDef object at 0x0000026463342920>, arguments=[Parameter(name='arg1', annotation='str')], docstring_text='    """\n    AZrojrltndflg lejkkjntgdf\n\n    Parameters\n    ----------\n    arg1: test\n\n    Returns\n    -------\n\n    """', sections=Sections(summary_text='    """\n    AZrojrltndflg lejkkjntgdf\n\n    ', param_delimiter='Parameters\n    ----------\n', parameters_text='    arg1: test\n\n    ', return_delimiter='Returns\n    -------\n', returns_text='\n    """'), start=4, length=11),
 Docstring(function=<ast.FunctionDef object at 0x0000026463342860>, arguments=[Parameter(name='arg1', annotation='list')], docstring_text='    """A function doing something\n\n\n    Second paragraph.\n    \n    Third paragraph\n    Parameters\n    ----------\n    arg1 (list): This does something\n\n    """', sections=Sections(summary_text='    """A function doing something\n\n\n    Second paragraph.\n    \n    Third paragraph\n    ', param_delimiter='Parameters\n   

In [42]:
[doc.sections.parameters_text for doc in docstrings]

['    arg1: test\n\n    ',
 '    arg1 (list): This does something\n\n    """',
 '    arg1\n\n    ',
 '    arg1: this does that\n\n    ',
 '    values: values to compute from\n    ref_id: delta values are computed with respect to that reference. It should be valid index or a list of valid\n     index from values.\n    -------\n\n    """',
 '    values: values to compute from\n    ref_id: delta values are computed with respect to that reference. It should be valid index or a list of valid\n     index from values.\n\n    ']